In [ ]:
import os
import numpy as np
import pandas as pd

Load csv

In [4]:
df_origin = pd.read_csv('C:\\school\\SchoolProgram\\NTUST_CSIE_DS\\DataSet\\acct_transaction.csv')
df_alert = pd.read_csv('C:\\school\\SchoolProgram\\NTUST_CSIE_DS\\DataSet\\acct_alert.csv')
print("交易資料維度:", df_origin.shape)
print("警示標籤維度:", df_alert.shape)
print("(Finish) Load Dataset.")

交易資料維度: (4435890, 10)
警示標籤維度: (1004, 2)
(Finish) Load Dataset.


轉出、轉入金額資訊

In [5]:
# 1. 'total_send/recv_amt': total amount sent/received by each acct
send = df_origin.groupby('from_acct')['txn_amt'].sum().rename('total_send_amt')
recv = df_origin.groupby('to_acct')['txn_amt'].sum().rename('total_recv_amt')

# 2. max, min, avg txn_amt for each account
max_send = df_origin.groupby('from_acct')['txn_amt'].max().rename('max_send_amt')
min_send = df_origin.groupby('from_acct')['txn_amt'].min().rename('min_send_amt')
avg_send = df_origin.groupby('from_acct')['txn_amt'].mean().rename('avg_send_amt')
var_send = df_origin.groupby('from_acct')['txn_amt'].var().rename('var_send_amt')
std_send = df_origin.groupby('from_acct')['txn_amt'].std().rename('std_send_amt')

max_recv = df_origin.groupby('to_acct')['txn_amt'].max().rename('max_recv_amt')
min_recv = df_origin.groupby('to_acct')['txn_amt'].min().rename('min_recv_amt')
avg_recv = df_origin.groupby('to_acct')['txn_amt'].mean().rename('avg_recv_amt')
var_recv = df_origin.groupby('to_acct')['txn_amt'].var().rename('var_recv_amt')
std_recv = df_origin.groupby('to_acct')['txn_amt'].std().rename('std_recv_amt')

互相轉帳的交易占比

In [9]:
total_send = df_origin.groupby('from_acct').size()
total_recv = df_origin.groupby('to_acct').size()

diff_send_account = df_origin.groupby('from_acct')['to_acct'].nunique()
diff_recv_account = df_origin.groupby('to_acct')['from_acct'].nunique()

mutual_send_ratio = (diff_send_account / total_send).rename('mutual_send_ratio')
mutual_recv_ratio = (diff_recv_account / total_recv).rename('mutual_recv_ratio')

使用交易通路種類

In [10]:
diffChannelType_send = df_origin.groupby('from_acct')['channel_type'].nunique().rename('diffChannelType_send')
diffChannelType_recv = df_origin.groupby('to_acct')['channel_type'].nunique().rename('diffChannelType_recv')

首次交易是否高轉帳金額

In [11]:
highAmountThreshold = df_origin['txn_amt'].quantile(0.8)

# 取得每個帳號的首筆送/收交易（依時間排序後取 first）
first_send = df_origin.sort_values(by=['from_acct', 'txn_time']).groupby('from_acct', as_index=False).first()
first_recv = df_origin.sort_values(by=['to_acct', 'txn_time']).groupby('to_acct', as_index=False).first()

# 轉成以 acct 為 index 的 Series，並判斷是否大於門檻
isFirstHighAmount_send = (first_send.set_index('from_acct')['txn_amt'] > highAmountThreshold).rename('isFirstHighAmount_send')
isFirstHighAmount_recv = (first_recv.set_index('to_acct')['txn_amt'] > highAmountThreshold).rename('isFirstHighAmount_recv')

# 補齊所有帳號（若帳號沒有首筆交易則為 False）
all_accts = pd.Index(pd.concat([df_origin['from_acct'], df_origin['to_acct']]).unique())
isFirstHighAmount_send = isFirstHighAmount_send.reindex(all_accts, fill_value=False)
isFirstHighAmount_recv = isFirstHighAmount_recv.reindex(all_accts, fill_value=False)

merge arrtibute

In [12]:
df_result = pd.concat([
    max_send, min_send, avg_send, var_send, std_send,
    max_recv, min_recv, avg_recv, var_recv, std_recv,
    send, recv, diffChannelType_send, diffChannelType_recv,
    isFirstHighAmount_send, isFirstHighAmount_recv, mutual_send_ratio, mutual_recv_ratio
], axis=1).fillna(0).reset_index()
df_result.rename(columns={'index': 'acct'}, inplace=True)

export csv

In [15]:
df_result.to_csv('C:\\school\\SchoolProgram\\NTUST_CSIE_DS\\XGBoost\\preprocessed_data.csv', index=False)
print("(Finish) Save preprocessed data to 'XGBoost\\preprocessed_data.csv'.")

(Finish) Save preprocessed data to 'XGBoost\preprocessed_data.csv'.
